# Construire un modèle de régression : préparer et visualiser les données

## **Régression linéaire pour les citrouilles - Leçon 2**
#### Introduction

Maintenant que vous disposez des outils nécessaires pour commencer à construire des modèles de machine learning avec Tidymodels et le Tidyverse, vous êtes prêt à poser des questions à vos données. Lorsque vous travaillez avec des données et appliquez des solutions de machine learning, il est crucial de savoir poser les bonnes questions pour exploiter pleinement le potentiel de votre jeu de données.

Dans cette leçon, vous apprendrez :

-   Comment préparer vos données pour la construction de modèles.

-   Comment utiliser `ggplot2` pour la visualisation des données.

La question à laquelle vous souhaitez répondre déterminera le type d'algorithmes de machine learning que vous utiliserez. Et la qualité de la réponse que vous obtiendrez dépendra fortement de la nature de vos données.

Voyons cela à travers un exercice pratique.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Illustration par @allison_horst</figcaption>


<!--![Illustration par \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.fr.jpg)<br>Illustration par \@allison_horst-->


## 1. Importer les données sur les citrouilles et invoquer le Tidyverse

Nous aurons besoin des packages suivants pour découper et analyser cette leçon :

-   `tidyverse` : Le [tidyverse](https://www.tidyverse.org/) est une [collection de packages R](https://www.tidyverse.org/packages) conçue pour rendre la science des données plus rapide, plus facile et plus amusante !

Vous pouvez les installer avec la commande suivante :

`install.packages(c("tidyverse"))`

Le script ci-dessous vérifie si vous avez les packages nécessaires pour compléter ce module et les installe pour vous si certains sont manquants.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Maintenant, lançons quelques packages et chargeons les [données](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) fournies pour cette leçon !


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Un rapide `glimpse()` montre immédiatement qu'il y a des valeurs manquantes et un mélange de chaînes de caractères (`chr`) et de données numériques (`dbl`). La colonne `Date` est de type caractère, et il y a aussi une colonne étrange appelée `Package` où les données sont un mélange de `sacks`, `bins` et d'autres valeurs. En réalité, les données sont un peu en désordre 😤.

En fait, il est assez rare de recevoir un jeu de données complètement prêt à l'emploi pour créer un modèle de machine learning directement. Mais ne vous inquiétez pas, dans cette leçon, vous apprendrez à préparer un jeu de données brut en utilisant des bibliothèques R standard 🧑‍🔧. Vous découvrirez également différentes techniques pour visualiser les données. 📈📊
<br>

> Un rappel : L'opérateur pipe (`%>%`) permet d'effectuer des opérations dans une séquence logique en passant un objet à une fonction ou une expression d'appel. Vous pouvez considérer l'opérateur pipe comme signifiant "et ensuite" dans votre code.


## 2. Vérifier les données manquantes

L'un des problèmes les plus courants auxquels les data scientists doivent faire face est celui des données incomplètes ou manquantes. R représente les valeurs manquantes ou inconnues avec une valeur sentinelle spéciale : `NA` (Non Disponible).

Alors, comment savoir si le tableau de données contient des valeurs manquantes ?
<br>
-   Une méthode simple serait d'utiliser la fonction de base R `anyNA`, qui retourne les objets logiques `TRUE` ou `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Super, il semble qu'il manque certaines données ! C'est un bon point de départ.

-   Une autre méthode consisterait à utiliser la fonction `is.na()` qui indique quels éléments individuels d'une colonne sont manquants avec une valeur logique `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

D'accord, c'est fait, mais avec un grand tableau de données comme celui-ci, il serait inefficace et pratiquement impossible de vérifier toutes les lignes et colonnes individuellement 😴.

-   Une méthode plus intuitive serait de calculer la somme des valeurs manquantes pour chaque colonne :


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Bien mieux ! Il manque des données, mais peut-être que cela n'aura pas d'importance pour la tâche en cours. Voyons ce que l'analyse supplémentaire peut révéler.

> En plus des ensembles impressionnants de packages et de fonctions, R dispose d'une très bonne documentation. Par exemple, utilisez `help(colSums)` ou `?colSums` pour en savoir plus sur la fonction.


## 3. Dplyr : Une grammaire pour manipuler les données

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Illustration par @allison_horst</figcaption>


<!--![Illustration par \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.fr.png)<br/>Illustration par \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), un package du Tidyverse, est une grammaire de manipulation de données qui offre un ensemble cohérent de verbes pour vous aider à résoudre les défis les plus courants en matière de manipulation de données. Dans cette section, nous allons explorer certains des verbes de dplyr !


#### dplyr::select()

`select()` est une fonction du package `dplyr` qui vous aide à choisir les colonnes à conserver ou à exclure.

Pour rendre votre data frame plus facile à manipuler, supprimez plusieurs de ses colonnes en utilisant `select()`, en ne gardant que les colonnes dont vous avez besoin.

Par exemple, dans cet exercice, notre analyse portera sur les colonnes `Package`, `Low Price`, `High Price` et `Date`. Sélectionnons ces colonnes.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` est une fonction du package `dplyr` qui vous permet de créer ou de modifier des colonnes, tout en conservant les colonnes existantes.

La structure générale de `mutate` est :

`data %>%   mutate(new_column_name = what_it_contains)`

Explorons `mutate` en utilisant la colonne `Date` pour effectuer les opérations suivantes :

1. Convertir les dates (actuellement de type caractère) en un format de mois (ce sont des dates américaines, donc le format est `MM/DD/YYYY`).

2. Extraire le mois des dates dans une nouvelle colonne.

En R, le package [lubridate](https://lubridate.tidyverse.org/) facilite le travail avec les données de type Date-heure. Utilisons donc `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` pour atteindre les objectifs ci-dessus. Nous pouvons supprimer la colonne Date, car nous n'en aurons plus besoin dans les opérations suivantes.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Youpi ! 🤩

Ensuite, créons une nouvelle colonne `Price`, qui représente le prix moyen d'une citrouille. Maintenant, prenons la moyenne des colonnes `Low Price` et `High Price` pour remplir la nouvelle colonne Price.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Ouiii !💪

"Mais attendez !", direz-vous après avoir parcouru rapidement l'ensemble des données avec `View(pumpkins)`, "Il y a quelque chose d'étrange ici !"🤔

Si vous regardez la colonne `Package`, les citrouilles sont vendues dans de nombreuses configurations différentes. Certaines sont vendues en mesures de `1 1/9 boisseau`, d'autres en mesures de `1/2 boisseau`, certaines à l'unité, d'autres au poids, et certaines dans de grandes caisses de largeurs variables.

Vérifions cela :


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Incroyable !👏

Les citrouilles semblent être très difficiles à peser de manière cohérente, alors filtrons-les en sélectionnant uniquement les citrouilles contenant le mot *bushel* dans la colonne `Package` et mettons cela dans un nouveau cadre de données `new_pumpkins`.


#### dplyr::filter() et stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html) : crée un sous-ensemble des données ne contenant que les **lignes** qui remplissent vos conditions, dans ce cas, les citrouilles ayant le mot *bushel* dans la colonne `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html) : détecte la présence ou l'absence d'un motif dans une chaîne de caractères.

Le package [`stringr`](https://github.com/tidyverse/stringr) fournit des fonctions simples pour les opérations courantes sur les chaînes de caractères.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Vous pouvez voir que nous avons réduit à environ 415 lignes de données contenant des citrouilles par boisseau.🤩
<br>


#### dplyr::case_when()

**Mais attendez ! Il y a encore une chose à faire**

Avez-vous remarqué que la quantité par boisseau varie selon les lignes ? Vous devez normaliser les prix afin d'afficher le prix par boisseau, et non par 1 1/9 ou 1/2 boisseau. Il est temps de faire quelques calculs pour standardiser cela.

Nous allons utiliser la fonction [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) pour *modifier* la colonne Price en fonction de certaines conditions. `case_when` permet de vectoriser plusieurs instructions `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nous pouvons maintenant analyser le prix par unité en fonction de leur mesure en boisseaux. Toute cette étude des boisseaux de citrouilles, cependant, montre à quel point il est `important` de `comprendre la nature de vos données` !

> ✅ Selon [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), le poids d'un boisseau dépend du type de produit, car il s'agit d'une mesure de volume. "Un boisseau de tomates, par exemple, est censé peser 56 livres... Les feuilles et les légumes verts occupent plus d'espace avec moins de poids, donc un boisseau d'épinards ne pèse que 20 livres." Tout cela est assez compliqué ! Ne nous embêtons pas à convertir les boisseaux en livres, et fixons plutôt les prix par boisseau. Toute cette étude des boisseaux de citrouilles, cependant, montre à quel point il est important de comprendre la nature de vos données !
>
> ✅ Avez-vous remarqué que les citrouilles vendues au demi-boisseau sont très chères ? Pouvez-vous deviner pourquoi ? Indice : les petites citrouilles coûtent beaucoup plus cher que les grosses, probablement parce qu'il y en a beaucoup plus par boisseau, étant donné l'espace inutilisé occupé par une grosse citrouille creuse pour tarte.


Maintenant, pour le simple plaisir de l'aventure 💁‍♀️, déplaçons également la colonne Mois à la première position, c'est-à-dire avant la colonne `Package`.

`dplyr::relocate()` est utilisé pour modifier la position des colonnes.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Bon travail !👌 Vous avez maintenant un ensemble de données propre et bien organisé sur lequel vous pouvez construire votre nouveau modèle de régression !


## 4. Visualisation des données avec ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infographie par Dasani Madipalli</figcaption>


<!--![Infographie par Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.fr.png){width="600"}-->

Il existe un *sage* dicton qui dit ceci :

> "Le simple graphique a apporté plus d'informations à l'esprit de l'analyste de données que tout autre outil." --- John Tukey

Une partie du rôle du data scientist est de démontrer la qualité et la nature des données avec lesquelles il travaille. Pour ce faire, il crée souvent des visualisations intéressantes, ou des graphiques, diagrammes et courbes, montrant différents aspects des données. De cette manière, il peut mettre en évidence visuellement des relations et des lacunes qui seraient autrement difficiles à détecter.

Les visualisations peuvent également aider à déterminer la technique d'apprentissage automatique la plus appropriée pour les données. Un nuage de points qui semble suivre une ligne, par exemple, indique que les données sont un bon candidat pour un exercice de régression linéaire.

R propose plusieurs systèmes pour créer des graphiques, mais [`ggplot2`](https://ggplot2.tidyverse.org/index.html) est l'un des plus élégants et des plus polyvalents. `ggplot2` vous permet de composer des graphiques en **combinant des composants indépendants**.

Commençons par un simple nuage de points pour les colonnes Price et Month.

Dans ce cas, nous commencerons avec [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), fournirons un jeu de données et une correspondance esthétique (avec [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), puis ajouterons des couches (comme [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) pour les nuages de points.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Est-ce un graphique utile 🤷 ? Y a-t-il quelque chose qui vous surprend ?

Ce n'est pas particulièrement utile, car tout ce qu'il fait est de montrer vos données sous forme de dispersion de points pour un mois donné.  


### **Comment le rendre utile ?**

Pour afficher des graphiques avec des données pertinentes, il est généralement nécessaire de regrouper les données d'une certaine manière. Par exemple, dans notre cas, calculer le prix moyen des citrouilles pour chaque mois fournirait davantage d'informations sur les tendances sous-jacentes de nos données. Cela nous amène à une autre fonctionnalité essentielle de **dplyr** :

#### `dplyr::group_by() %>% summarize()`

L'agrégation par groupes en R peut être facilement réalisée avec :

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` modifie l'unité d'analyse, passant de l'ensemble du jeu de données à des groupes individuels, comme par exemple par mois.

-   `dplyr::summarize()` crée un nouveau tableau de données avec une colonne pour chaque variable de regroupement et une colonne pour chaque statistique récapitulative que vous avez spécifiée.

Par exemple, nous pouvons utiliser `dplyr::group_by() %>% summarize()` pour regrouper les citrouilles en fonction de la colonne **Month**, puis calculer le **prix moyen** pour chaque mois.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Succinct!✨

Les caractéristiques catégoriques comme les mois sont mieux représentées à l'aide d'un diagramme en barres 📊. Les couches responsables des graphiques en barres sont `geom_bar()` et `geom_col()`. Consultez `?geom_bar` pour en savoir plus.

Créons-en un !


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Voici une visualisation des données plus utile ! Cela semble indiquer que le prix le plus élevé des citrouilles se produit en septembre et octobre. Est-ce que cela correspond à vos attentes ? Pourquoi ou pourquoi pas ?

Félicitations pour avoir terminé la deuxième leçon 👏 ! Vous avez préparé vos données pour la construction du modèle, puis découvert davantage d'informations grâce aux visualisations !



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de faire appel à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
